<a href="https://colab.research.google.com/github/alesaccoia/IULM_DDM2324_Notebooks/blob/main/18_BTYD_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install lifetimes

In [ ]:
from lifetimes.datasets import load_cdnow_summary
data = load_cdnow_summary(index_col=[0])

print(data.head())

Vediamo ora la dimensionalitá del dataset:

In [ ]:
data.shape

In [ ]:
data.describe()

# Utilizzo del Modello

In [ ]:
from lifetimes import BetaGeoFitter

# similar API to scikit-learn and lifelines.
bgf = BetaGeoFitter(penalizer_coef=0.0)
bgf.fit(data['frequency'], data['recency'], data['T'])
print(bgf)

Una volta che il modello é stato lanciato, si possono visualizzare molti grafici interessanti.

Considera il seguente scenario: un cliente ha acquistato da te ogni giorno per tre settimane di fila, e non abbiamo più sentito nulla da loro da mesi. Quali sono le probabilità che siano ancora "attivi"? Piuttosto basse. D'altra parte, un cliente che storicamente acquista da te una volta al trimestre e ha acquistato l'ultimo trimestre, è probabile che sia ancora attivo. Possiamo visualizzare questa relazione utilizzando la matrice Frequenza/Tempo trascorso, che calcola il numero previsto di transazioni che un cliente artificiale effettuerà nel prossimo periodo di tempo, in base alla sua recente attività (età all'ultimo acquisto) e alla frequenza (il numero di transazioni ripetute che ha effettuato).

In [ ]:
from lifetimes.plotting import plot_frequency_recency_matrix

plot_frequency_recency_matrix(bgf)


Possiamo notare che se un cliente ha acquistato 25 volte da te e il suo ultimo acquisto è stato quando avevano 35 settimane di età (considerando che l'individuo ha 35 settimane di età), allora sono il tuo miglior cliente (in basso a destra). I tuoi clienti meno attivi si trovano nell'angolo in alto a destra: hanno acquistato molto rapidamente e non li abbiamo visti da settimane.

C'è anche quella bellissima "coda" intorno a (5,25). Rappresenta il cliente che acquista raramente, ma l'abbiamo visto di recente, quindi potrebbero acquistare di nuovo - non siamo sicuri se siano inattivi o semplicemente tra un acquisto e l'altro.

Un'altra matrice interessante da esaminare è la probabilità di essere ancora attivi:

In [ ]:
from lifetimes.plotting import plot_probability_alive_matrix

plot_probability_alive_matrix(bgf)

In [ ]:
t = 1
data['predicted_purchases'] = bgf.conditional_expected_number_of_purchases_up_to_time(t, data['frequency'], data['recency'], data['T'])
data.sort_values(by='predicted_purchases').tail(5)

In [ ]:
from lifetimes.datasets import load_transaction_data
from lifetimes.utils import summary_data_from_transaction_data

transaction_data = load_transaction_data()
print(transaction_data.head())

In [ ]:
transaction_data.date.min(),transaction_data.date.max()

In [ ]:
summary = summary_data_from_transaction_data(transaction_data, 'id', 'date', observation_period_end='2014-12-31')
print(summary.head())